<a href="https://colab.research.google.com/github/SeanWangreal/trading-collab/blob/main/Finlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install finlab > log.txt
!pip install matplotlib > log2.txt

In [ ]:
from finlab import data
import pandas as pd

# psd = data.get('price:收盤價')

# print(psd['3224']['2007-04-23'])
# df = pd.DataFrame(psd)
# print(df)

# transposed_df = df.T
# print(data.get('price:成交股數').keys())
# data.get('price:成交股數')
pd.DataFrame(data.get('monthly_revenue:上月比較增減(%)'))


symbol,000116,000930,000960,0009A0,1101,1102,1103,1104,1107,1108,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2005-02-10,NaN,NaN,NaN,NaN,5.00,9.00,4.00,-2.00,28.0,8.00,...,23.00,-57.00,4.00,-7.00,-6.00,-2.00,-37.00,NaN,-18.00,NaN
2005-03-10,NaN,NaN,NaN,NaN,-42.00,-39.00,-42.00,-41.00,-15.0,-52.00,...,3.00,0.00,-14.00,-27.00,-20.00,-47.00,2.00,NaN,-3.00,NaN
2005-04-11,NaN,NaN,NaN,NaN,59.00,45.00,49.00,45.00,154.0,100.00,...,14.00,2.00,7.00,-7.00,12.00,138.00,36.00,NaN,10.00,NaN
2005-05-10,NaN,NaN,NaN,NaN,-3.00,-9.00,0.00,2.00,-44.0,0.00,...,-9.00,-33.00,-18.00,10.00,-1.00,-31.00,-13.00,NaN,6.00,NaN
2005-06-10,NaN,NaN,NaN,NaN,0.00,6.00,-5.00,2.00,1.0,-8.00,...,5.00,16.00,5.00,4.00,21.00,-9.00,2.00,NaN,-3.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-10,NaN,NaN,NaN,NaN,-14.81,-11.27,-6.56,-8.49,NaN,-5.05,...,-40.92,10.09,10.73,12.28,9.70,8.72,133.26,-4.76,20.22,-3.12
2024-06-11,NaN,NaN,NaN,NaN,19.21,13.59,2.39,8.21,NaN,2.10,...,-6.59,-12.78,1.03,-20.43,3.33,12.06,-39.38,-1.25,-4.82,32.17
2024-07-10,NaN,NaN,NaN,NaN,-1.44,5.86,5.12,-10.87,NaN,-18.60,...,-7.23,1.70,-60.72,35.11,17.88,-12.72,-16.55,-0.26,-0.85,-10.35


月營收增長率策略

In [ ]:
from finlab import data
import pandas as pd
import math

from datetime import datetime, timedelta
from tabulate import tabulate
import matplotlib.pyplot as plt


PRICE_DF = pd.DataFrame(data.get('price:收盤價'));
monthRate = pd.DataFrame(data.get('monthly_revenue:上月比較增減(%)'))
# dealStockNum = pd.DataFrame(data.get('price:成交股數'))
dealStockNum = pd.DataFrame(data.get('price:成交金額'))

TRANS_MONTH_RATE = monthRate.T
INVEST_FUND = 1000000;
updatedInvestFund = INVEST_FUND;
HANDING_FEE_RATE = (0.1425*0.01);
TAX_RATE = (0.3*0.01)
STOCK_NUM = 10;
# 金額投入比率，會動態調動
INVEST_PERSENT = 1;

def getPrice(stock, originDate, date):
  time_difference = date - originDate
  # 如果與原訂的(買/賣)時間相差10天就當作不買或當他下市
  if time_difference.days == 10:
    return 0;
  formatted_date = date.strftime("%Y-%m-%d")
  if formatted_date in PRICE_DF[stock].keys():
    return PRICE_DF[stock][formatted_date];
  # 如果資料找不到該天就回傳-1
  return -1;

# 傳入資料齊全的所有股票，並依照上個月營收成長率排序，回傳前10高的
def getInvestTarget(allStockInfo):
  orderedInfo = sorted(allStockInfo, key=lambda x: x['rate'], reverse=True)
  return orderedInfo[:STOCK_NUM];

# 取得手續費
def getHandingFee(amount):
  return round(amount * HANDING_FEE_RATE);

# 取得交易稅
def getTax(amount):
  return round(amount * TAX_RATE);

# 買投資組合
def buy(buyTarget, delayTradeDay):
  # 各個投資標的平均投資金額
  amount = (updatedInvestFund*INVEST_PERSENT) // len(buyTarget)
  for target in buyTarget:
    originDate = target["date"] + timedelta(days=delayTradeDay);
    targetDate = target["date"] + timedelta(days=delayTradeDay);
    stock = target["stock"];
    price = getPrice(stock, originDate, targetDate)
    # 資料若找不到就多找幾次
    while (math.isnan(price) or price == -1):
      targetDate += timedelta(days=1);
      price = getPrice(stock, originDate, targetDate);

    target["buyPrice"] = price;
    unit = 0 if price == 0 else amount // price;
    target["unit"] = unit;
    buyAmount = round(price * unit);
    handlingFee = getHandingFee(buyAmount) if getHandingFee(buyAmount) < 20 else 20
    target["cost"] = buyAmount + handlingFee
  return buyTarget;

def sell(sellTarget, month):
  # print('-------begin sell-------')
  revenue = 0;
  for target in sellTarget:
    originDate = month;
    targetDate = month;
    sellPrice = getPrice(target["stock"], originDate, targetDate)
    while (math.isnan(sellPrice) or sellPrice == -1):
      targetDate += timedelta(days=1);
      sellPrice = getPrice(target["stock"], originDate, targetDate);
    target["sellDate"] = targetDate
    target["sellPrice"] = sellPrice
    sellAmount = round(sellPrice * target["unit"]);
    handlingFee = getHandingFee(sellAmount) if getHandingFee(sellAmount) < 20 else 20
    tax = getTax(sellAmount);
    sellAmount = sellAmount - (handlingFee + tax);
    target["sellAmount"] = sellAmount;
    # 最終賣的總金額 - 上個月買的總金額
    profit = (sellAmount - target["cost"])
    target["profit"] = profit
    revenue += profit;
  # print(sellTarget)
  table = sellTarget;
  # Print the table
  print(tabulate(table,headers="keys", tablefmt="grid"))
  # print('sell date : '+ str(month))
  # print('-------end sell-------')
  return revenue;

allMonth = TRANS_MONTH_RATE.keys()
totalRevenue = 0;
buyMonth = '';
lastBuyTarget = {};
# startTime = pd.Timestamp("2007-04-23 00:00:00");
startTime = pd.Timestamp("2014-09-01 00:00:00");
endTime = pd.Timestamp("2024-09-11 00:00:00");
count = 0;
# 新舊高峰
oldHigh = updatedInvestFund;
newHigh = updatedInvestFund;

mdd = 0;
fundPriceInterval = [INVEST_FUND];
fundPriceAllInterval = [INVEST_FUND];
tradeTime = [startTime.strftime("%y/%m")];
mddList = [];
# 股票投入金額大於前天市場交易金額倍數，目前3倍最好
saveMuiple = 1.5;
saveBonus = 0;
for month in allMonth:
  if month >= startTime:
    oneMonthData = TRANS_MONTH_RATE[month];
    stocklist = [];
    # 清洗stock排除不符合條件的 ⬇⬇⬇
    for stock in oneMonthData.keys():
      if stock in dealStockNum.keys():
        datetime_index = month.date()
        dealStockNo = dealStockNum[stock][str(datetime_index)];
        buyOneStockPay = ((updatedInvestFund*INVEST_PERSENT)/STOCK_NUM);

        if not math.isnan(dealStockNo) and dealStockNo > ((buyOneStockPay * saveMuiple) + saveBonus):
          if stock in PRICE_DF.keys():
            rate = oneMonthData[stock];
            if math.isnan(rate):
              rate = -100000;
            stockInfo = {'stock':stock,'rate':rate, 'date':month}
            stocklist.append(stockInfo)
        else:
          pass;
    # 清洗stock排除不符合條件的 ⬆⬆⬆
    buyTarget = getInvestTarget(stocklist)

    delayBuyDay = 0;
    if (buyMonth == ''):
      lastBuyTarget = buy(buyTarget, delayBuyDay);
    else:
      count += 1;
      revenue = sell(lastBuyTarget,month);
      print("該次交易總收益: "+str(revenue))
      # 投入金額會依照收益再投資，該值等同於基金價格
      updatedInvestFund += revenue;
      fromTopToNowDDRate = (oldHigh - updatedInvestFund) / oldHigh * 100
      # print(fromTopToNowDDRate)
      # 調整資金參數(目前無效) ⬇⬇⬇
      # if fromTopToNowDDRate >= 30:
      #   saveMuiple = 3;
      #   INVEST_PERSENT = 1;
      # elif fromTopToNowDDRate <= 0:
      #   saveMuiple = 3;
      #   INVEST_PERSENT = 1;
      # else:
      #   saveMuiple = 3;
      #   INVEST_PERSENT = 1;
      # 調整資金參數(目前無效) ⬆⬆⬆
      tradeTime.append(month.strftime("%y/%m"));
      fundPriceInterval.append(updatedInvestFund);
      fundPriceAllInterval.append(updatedInvestFund);
      print('updatedInvestFund : ' +str(updatedInvestFund))
      # 計算新高峰及新低谷
      if updatedInvestFund > oldHigh:
        newHigh = updatedInvestFund;
        dd = ((oldHigh - min(fundPriceInterval)) / oldHigh) * 100;
        # print('oldHigh : ' +str(oldHigh))
        # print('newLow : ' +str(min(fundPriceInterval)))
        # print('newHigh : ' +str(newHigh))
        # print('DD : ' +str(dd))
        if dd > 0:
          mddObj = {'MDD':dd,'DATE':str(month)};
          mddList.append(mddObj)
        if dd > mdd:
          mdd = dd;
          print('MDD : ' +str(dd) + 'DATE : '+ str(month));
        oldHigh = newHigh;
        fundPriceInterval = [updatedInvestFund];
      totalRevenue += revenue;
      lastBuyTarget = buy(buyTarget, delayBuyDay);

    buyMonth = month;
    if month >= endTime:
      break;

years = count / 12;
print('total counts : ' + str(count))
print('total years : ' + str(years))
print('total revenues : ' + str(totalRevenue))
print('revenue per year: ' + str(totalRevenue/years))
print('平均年報酬率 : ' + str((totalRevenue/INVEST_FUND/years)*100)+'%')
print('最大回撤率(MDD) : ' + str(mdd) + '%')
print(mddList)

# Plot
plt.figure(figsize=(100, 60))
plt.plot(tradeTime, fundPriceAllInterval, marker="o", linestyle="-", color="blue", label="Price over Time")
plt.xlabel("Time", fontsize=12)
plt.ylabel("Fund Price", fontsize=12)
plt.title("Time vs. Fund Price", fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)
plt.tight_layout()

# Save and display the plot
# file_path = "/mnt/data/Time_vs_Price.png"
# plt.savefig(file_path)
plt.show()

+---------+----------+---------------------+------------+--------+--------+---------------------+-------------+--------------+----------+
|   stock |     rate | date                |   buyPrice |   unit |   cost | sellDate            |   sellPrice |   sellAmount |   profit |
+=========+==========+=====================+============+========+========+=====================+=============+==============+==========+
|    2527 | 34065.7  | 2014-09-10 00:00:00 |      18.45 |   5420 | 100019 | 2014-10-13 00:00:00 |       17.2  |        92924 |    -7095 |
+---------+----------+---------------------+------------+--------+--------+---------------------+-------------+--------------+----------+
|    6219 |  3951.48 | 2014-09-10 00:00:00 |      18.7  |   5347 | 100009 | 2014-10-13 00:00:00 |       17.15 |        91406 |    -8603 |
+---------+----------+---------------------+------------+--------+--------+---------------------+-------------+--------------+----------+
|    3126 |  1131.3  | 2014-09-10 

0050 10y -> MDD 35% IRR 15%

此策略10y

 1倍   -> MDD 43% IRR 35%

 2倍   -> MDD 33% IRR 34%

 3倍   -> MDD 30% IRR 50%

 3.5倍   -> MDD 30% IRR 50%

 4倍   -> MDD 30% IRR 48%

In [ ]:
from finlab import data
import pandas as pd
import math
from datetime import datetime, timedelta
from tabulate import tabulate

# PRICE_DF = pd.DataFrame(data.get('price:收盤價'));
# monthRate = pd.DataFrame(data.get('monthly_revenue:上月比較增減(%)'))
# data.get('company_basic_info')
# print(data.get('company_basic_info')['實收資本額(元)'])
# 股票市值
# data.get('etl:market_value')
stockMarketVal = data.get('etl:market_value').T;
stockMarketVal
print(stockMarketVal.keys());
# print(stockMarketVal['2018-12-28'].keys())
print(stockMarketVal['2018-12-28']['2330'])
print(data.get('company_basic_info')['實收資本額(元)'])

DatetimeIndex(['2013-04-19', '2013-04-22', '2013-04-23', '2013-04-24',
               '2013-04-25', '2013-04-26', '2013-04-29', '2013-04-30',
               '2013-05-02', '2013-05-03',
               ...
               '2018-12-18', '2018-12-19', '2018-12-20', '2018-12-21',
               '2018-12-22', '2018-12-24', '2018-12-25', '2018-12-26',
               '2018-12-27', '2018-12-28'],
              dtype='datetime64[ns]', name='date', length=1503, freq=None)
5847300802750.0
0       77511817420
1       35465628810
2        7902474590
3        6866818160
4        4047380490
           ...     
2183      256320150
2184     2805781150
2185     1365000000
2186      358500000
2187    26204136000
Name: 實收資本額(元), Length: 2188, dtype: int64


In [ ]:
import pandas as pd
import numpy as np

df = {'a':[1,np.nan,np.nan,2,3,4,5,np.nan]};

df = pd.DataFrame(data=df);
mean = df.rolling(window=3, min_periods=2).mean()
mean

data.get('price:成交金額')

Daily usage: 168.4 / 500 MB - price:成交金額


symbol,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2007-04-23,992070.0,4.955742e+08,25746690.0,3993530.0,NaN,NaN,NaN,NaN,NaN,NaN,...,22411245.0,765924013.0,360397.0,33760335.0,105950.0,18144214.0,85132460.0,NaN,19369041.0,37477098.0
2007-04-24,1783740.0,1.436573e+08,9432550.0,8507700.0,NaN,NaN,NaN,NaN,NaN,NaN,...,57293100.0,467579125.0,505770.0,21410685.0,118150.0,7119720.0,66706510.0,NaN,12465642.0,79594212.0
2007-04-25,856220.0,7.095020e+07,4526020.0,7451240.0,NaN,NaN,NaN,NaN,NaN,NaN,...,73244602.0,285633162.0,576233.0,13591700.0,211650.0,13436443.0,60495998.0,NaN,17361810.0,48637198.0
2007-04-26,978560.0,3.667542e+08,3613274.0,5279168.0,NaN,NaN,NaN,NaN,NaN,NaN,...,35536158.0,224825582.0,3504327.0,22601824.0,185200.0,6489150.0,40600570.0,NaN,12406728.0,85060188.0
2007-04-27,248300.0,6.147500e+07,3272450.0,5625203.0,NaN,NaN,NaN,NaN,NaN,NaN,...,25720407.0,447237567.0,5488869.0,13128987.0,317850.0,6118400.0,48087282.0,NaN,3683259.0,83982200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-02,NaN,2.749829e+09,11554644.0,244856502.0,1358995.0,NaN,2308134.0,3.082997e+09,2294537.0,NaN,...,3219166.0,85866380.0,5565690.0,178231.0,1532924.0,2687038.0,3636003.0,222397378.0,132314.0,820442.0
2025-01-03,NaN,1.461802e+09,7574637.0,197779316.0,975509.0,NaN,2287024.0,1.962177e+09,2449640.0,NaN,...,1710564.0,68079869.0,4948487.0,1081392.0,2002696.0,2636462.0,28235179.0,279426578.0,357452.0,1322979.0
2025-01-06,NaN,5.292043e+09,14800009.0,388451250.0,3077641.0,NaN,7873173.0,2.163743e+09,4863316.0,NaN,...,1656712.0,105250242.0,3534566.0,634578.0,1774957.0,2349565.0,16539479.0,259912598.0,182039.0,310628.0
